In [10]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from hyperopt.pyll import scope
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, train_test_split
import sys
sys.path.append('./TuRBO')  # Adjust this path
import torch
from turbo import Turbo1


In [ ]:
df = pd.read_csv('C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv', sep=',')
df.head()


,Transaction Hash_len,Original_len,signature_len,From_len,To_len,sender_len,paymaster_len,Txn Fee,Gas Used,logIndex,actualGasCost,actualGasUsed,nonce,success,Blockno,DateTime_ts
0,32,964,4,20,20,20,20,0.022033,398741,245,2.190000e+16,397164,0.0,1,17066994,1681740540
1,32,868,4,20,20,20,20,0.004362,87702,231,4.280000e+15,86113,1.0,1,17067000,1681740600
2,32,868,4,20,20,20,20,0.003971,87714,273,3.900000e+15,86125,2.0,1,17067009,1681740720
3,32,1188,4,20,20,20,20,0.008673,161702,429,8.650000e+15,161161,3.0,1,17078992,1681887000
4,32,1188,4,20,20,20,20,0.006445,127502,393,6.420000e+15,126973,4.0,1,17079029,1681887420


In [ ]:
# ===== 0) Imports =====
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler

from catboost import CatBoostRegressor

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll import scope

# ===== 1) Load data ===== 
csv_path = r"C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv"
df = pd.read_csv(csv_path)

features = [
    'Transaction Hash_len', 'Original_len', 'signature_len',
    'From_len', 'To_len', 'sender_len', 'paymaster_len',
    'Txn Fee', 'logIndex', 'actualGasCost',
    'actualGasUsed', 'nonce', 'success', 'Blockno', 'DateTime_ts'
]
X = df[features].reset_index(drop=True).astype(float)
y = df['Gas Used'].astype(float).reset_index(drop=True)

# (Tuỳ chọn) log-transform target nếu skew mạnh
USE_LOG_TARGET = False
y_trainable = np.log1p(y) if USE_LOG_TARGET else y.copy()

# ===== 2) Search space =====
def base_space():
    return {
        'depth': scope.int(hp.quniform('depth', 4, 10, 1)),
        'l2_leaf_reg': hp.loguniform('l2_leaf_reg', np.log(1e-2), np.log(1e2)),
        'iterations': scope.int(hp.quniform('iterations', 400, 2000, 50)),
        'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
        'bootstrap_type': hp.choice('bootstrap_type', ['Bayesian', 'Bernoulli']),
        'subsample': hp.uniform('subsample', 0.6, 1.0),
        'bagging_temperature': hp.uniform('bagging_temperature', 0.0, 1.0),
        'random_strength': hp.uniform('random_strength', 0.0, 5.0),
        'rsm': hp.uniform('rsm', 0.6, 1.0),
        'leaf_estimation_iterations': scope.int(hp.quniform('leaf_estimation_iterations', 1, 20, 1)),
    }

def trust_space(center, delta=0.25):
    def clamp(v, lo, hi): 
        return float(max(lo, min(hi, v)))
    return {
        'depth': scope.int(hp.quniform(
            'depth',
            max(4, int(center['depth']) - 1),
            min(10, int(center['depth']) + 1), 1
        )),
        'l2_leaf_reg': hp.loguniform(
            'l2_leaf_reg',
            np.log(clamp(center['l2_leaf_reg'] * (1 - delta), 1e-2, 1e2)),
            np.log(clamp(center['l2_leaf_reg'] * (1 + delta), 1e-2, 1e2))
        ),
        'iterations': scope.int(hp.quniform(
            'iterations',
            clamp(center['iterations'] - 300, 200, 3000),
            clamp(center['iterations'] + 300, 200, 3000), 50
        )),
        'learning_rate': hp.loguniform(
            'learning_rate',
            np.log(clamp(center['learning_rate'] * (1 - delta), 0.005, 0.3)),
            np.log(clamp(center['learning_rate'] * (1 + delta), 0.005, 0.3))
        ),
        'bootstrap_type': hp.choice('bootstrap_type', ['Bayesian', 'Bernoulli']),
        'subsample': hp.uniform(
            'subsample',
            clamp(center.get('subsample', 0.8) - delta, 0.5, 1.0),
            clamp(center.get('subsample', 0.8) + delta, 0.5, 1.0)
        ),
        'bagging_temperature': hp.uniform(
            'bagging_temperature',
            clamp(center.get('bagging_temperature', 0.5) - delta, 0.0, 1.0),
            clamp(center.get('bagging_temperature', 0.5) + delta, 0.0, 1.0)
        ),
        'random_strength': hp.uniform(
            'random_strength',
            clamp(center.get('random_strength', 1.0) - 2*delta, 0.0, 10.0),
            clamp(center.get('random_strength', 1.0) + 2*delta, 0.0, 10.0)
        ),
        'rsm': hp.uniform(
            'rsm',
            clamp(center.get('rsm', 0.8) - delta, 0.5, 1.0),
            clamp(center.get('rsm', 0.8) + delta, 0.5, 1.0)
        ),
        'leaf_estimation_iterations': scope.int(hp.quniform(
            'leaf_estimation_iterations',
            max(1, int(center.get('leaf_estimation_iterations', 10)) - 5),
            min(32, int(center.get('leaf_estimation_iterations', 10)) + 5), 1
        )),
    }

# ===== 3) Cross-validation objective (scaled per fold cho cả X & y) =====
kf = KFold(n_splits=10, shuffle=True, random_state=42)

def cv_metrics(params):
    p = params.copy()
    p['depth'] = int(p['depth'])
    p['iterations'] = int(p['iterations'])
    p['leaf_estimation_iterations'] = int(p['leaf_estimation_iterations'])
    p['subsample'] = float(p['subsample'])
    p['rsm'] = float(p['rsm'])

    rmse_list, mse_list, mae_list, r2_list = [], [], [], []

    for tr_idx, va_idx in kf.split(X):
        X_tr, X_va = X.iloc[tr_idx].values, X.iloc[va_idx].values
        y_tr, y_va = y_trainable.iloc[tr_idx].values, y_trainable.iloc[va_idx].values

        # scale per fold
        scaler_X = MinMaxScaler()
        X_tr_s = scaler_X.fit_transform(X_tr)
        X_va_s = scaler_X.transform(X_va)

        scaler_y = MinMaxScaler()
        y_tr_s = scaler_y.fit_transform(y_tr.reshape(-1, 1)).ravel()
        y_va_s = scaler_y.transform(y_va.reshape(-1, 1)).ravel()

        kwargs = dict(
            loss_function='RMSE',
            eval_metric='R2',
            iterations=p['iterations'],
            learning_rate=float(p['learning_rate']),
            depth=p['depth'],
            l2_leaf_reg=float(p['l2_leaf_reg']),
            random_strength=float(p['random_strength']),
            rsm=float(p['rsm']),
            leaf_estimation_iterations=p['leaf_estimation_iterations'],
            random_seed=42,
            verbose=False,
            allow_writing_files=False
        )
        if p['bootstrap_type'] == 'Bayesian':
            kwargs.update(bootstrap_type='Bayesian',
                          bagging_temperature=float(p['bagging_temperature']))
        else:
            kwargs.update(bootstrap_type='Bernoulli',
                          subsample=float(p['subsample']))

        model = CatBoostRegressor(**kwargs)
        model.fit(X_tr_s, y_tr_s, eval_set=(X_va_s, y_va_s),
                  use_best_model=True, early_stopping_rounds=200)

        preds_s = model.predict(X_va_s)

        mse  = mean_squared_error(y_va_s, preds_s)
        rmse = float(np.sqrt(mse))
        mae  = float(mean_absolute_error(y_va_s, preds_s))
        r2   = float(r2_score(y_va_s, preds_s))

        rmse_list.append(rmse)
        mse_list.append(mse)
        mae_list.append(mae)
        r2_list.append(r2)

    return {
        'rmse': rmse_list,
        'mse' : mse_list,
        'mae' : mae_list,
        'r2'  : r2_list,
        'mean_r2': float(np.mean(r2_list))
    }

def objective(params):
    out = cv_metrics(params)
    if out is None:
        return {'loss': 1e9, 'status': STATUS_OK}
    return {
        'loss': float(1.0 - out['mean_r2']),
        'status': STATUS_OK,
        'metrics': out,
        'params': params
    }

# ===== 4) Bayesian Optimization + Trust Region =====
TOTAL_EVALS = 30   # tăng lên 60/120 nếu muốn
STEP = 30
TOP_K = 10
trials = Trials()
rng = np.random.default_rng(42)

for i in range(0, TOTAL_EVALS, STEP):
    if i == 0:
        space = base_space()
    else:
        past = [t for t in trials.trials if 'loss' in t['result']]
        past = sorted(past, key=lambda t: t['result']['loss'])
        topk = past[: min(TOP_K, len(past))]
        center = {}
        for k in ['depth','l2_leaf_reg','iterations','learning_rate','subsample',
                  'bagging_temperature','random_strength','rsm','leaf_estimation_iterations']:
            vals = [t['result']['params'][k] for t in topk if k in t['result']['params']]
            center[k] = float(np.mean(vals)) if len(vals) else {
                'learning_rate':0.05,'subsample':0.8,'bagging_temperature':0.5,
                'random_strength':1.0,'rsm':0.8,'leaf_estimation_iterations':10
            }.get(k, 8.0)
        space = trust_space(center, delta=0.25)

    fmin(fn=objective, space=space, algo=tpe.suggest,
         max_evals=i + STEP, trials=trials, rstate=rng)

# ===== 5) Best trial =====
best_trial = min(trials.trials, key=lambda t: t['result']['loss'])
best_r2_scaled = 1.0 - best_trial['result']['loss']
best_params = best_trial['result']['params']

print("\n🏆 Best R² (CV mean, scaled y):", best_r2_scaled)
print("🔧 Best params:", best_params)

# ===== 6) Re-evaluate BEST PARAMS per fold =====
mse_scores, mae_scores, rmse_scores, r2_scores = [], [], [], []
p = dict(best_params)
p['depth'] = int(p['depth'])
p['iterations'] = int(p['iterations'])
p['leaf_estimation_iterations'] = int(p['leaf_estimation_iterations'])

for fold, (tr_idx, va_idx) in enumerate(kf.split(X), start=1):
    X_tr, X_va = X.iloc[tr_idx].values, X.iloc[va_idx].values
    y_tr, y_va = y_trainable.iloc[tr_idx].values, y_trainable.iloc[va_idx].values

    scaler_X = MinMaxScaler()
    X_tr_s = scaler_X.fit_transform(X_tr)
    X_va_s = scaler_X.transform(X_va)

    scaler_y = MinMaxScaler()
    y_tr_s = scaler_y.fit_transform(y_tr.reshape(-1, 1)).ravel()
    y_va_s = scaler_y.transform(y_va.reshape(-1, 1)).ravel()

    params = dict(
        loss_function='RMSE',
        eval_metric='R2',
        iterations=p['iterations'],
        learning_rate=float(p['learning_rate']),
        depth=p['depth'],
        l2_leaf_reg=float(p['l2_leaf_reg']),
        random_strength=float(p['random_strength']),
        rsm=float(p['rsm']),
        leaf_estimation_iterations=p['leaf_estimation_iterations'],
        random_seed=42,
        verbose=False,
        allow_writing_files=False,
        use_best_model=True
    )
    if p['bootstrap_type'] == 'Bayesian':
        params.update(bootstrap_type='Bayesian',
                      bagging_temperature=float(p['bagging_temperature']))
    else:
        params.update(bootstrap_type='Bernoulli',
                      subsample=float(p['subsample']))

    model = CatBoostRegressor(**params)
    model.fit(X_tr_s, y_tr_s, eval_set=(X_va_s, y_va_s),
              early_stopping_rounds=200)
    preds_s = model.predict(X_va_s)

    mse  = mean_squared_error(y_va_s, preds_s)
    rmse = np.sqrt(mse)
    mae  = mean_absolute_error(y_va_s, preds_s)
    r2   = r2_score(y_va_s, preds_s)

    mse_scores.append(mse)
    mae_scores.append(mae)
    rmse_scores.append(rmse)
    r2_scores.append(r2)

    print(f"Fold {fold} - (scaled) "
          f"MSE: {mse:.6f} | MAE: {mae:.6f} | RMSE: {rmse:.6f} | R²: {r2:.6f}")

print("\n=== Kết quả trung bình (SCALED) ===")
print(f"Avg MSE : {np.mean(mse_scores):.6f}")
print(f"Avg MAE : {np.mean(mae_scores):.6f}")
print(f"Avg RMSE: {np.mean(rmse_scores):.6f}")
print(f"Avg R²  : {np.mean(r2_scores):.6f}")

In [ ]:
# ================== CONFIG ==================
EPOCHS = 20
K = 10
SEED = 42
np.random.seed(SEED)

# ================== BEST PARAMS (từ Hyperopt) ==================
best_params = {
    'bagging_temperature': 0.03808810504137822,
    'bootstrap_type': 'Bayesian',
    'depth': 9,
    'iterations': 1150,
    'l2_leaf_reg': 3.157088896921137,
    'leaf_estimation_iterations': 18,
    'learning_rate': 0.04688120366273992,
    'random_strength': 2.0533400767421526,
    'rsm': 0.683305874834135,
    'subsample': 0.908911282734798
}

# ÉP KIỂU & LÀM SẠCH PARAMS
p = dict(best_params)
p['depth'] = int(p.get('depth', 8))
p['iterations'] = int(p.get('iterations', 5000))
p['leaf_estimation_iterations'] = int(p.get('leaf_estimation_iterations', 10))
for k in ['learning_rate','l2_leaf_reg','random_strength','bagging_temperature','rsm','subsample']:
    if k in p:
        p[k] = float(p[k])
btype = p.get('bootstrap_type', 'Bayesian')

def build_kwargs(seed_offset: int):
    kwargs = dict(
        loss_function='RMSE',
        eval_metric='R2',
        iterations=p['iterations'],
        learning_rate=p['learning_rate'],
        depth=p['depth'],
        l2_leaf_reg=p['l2_leaf_reg'],
        random_strength=p['random_strength'],
        leaf_estimation_iterations=p['leaf_estimation_iterations'],
        random_seed=SEED + seed_offset,
        allow_writing_files=False,
        verbose=False,
    )
    # rsm: một số phiên bản CatBoost không hỗ trợ → sẽ thử/loại ở dưới
    if 'rsm' in p:
        kwargs['rsm'] = p['rsm']

    if btype == 'Bayesian':
        kwargs.update(bootstrap_type='Bayesian',
                      bagging_temperature=p.get('bagging_temperature', 1.0))
    else:
        kwargs.update(bootstrap_type='Bernoulli',
                      subsample=p.get('subsample', 0.8))
    return kwargs

# ================== 1) LOAD & PREP ==================
df = pd.read_csv(r'C:/Users/Multiplexon/Desktop/data/2/total_selected_augmented.csv')

features = [
    'Transaction Hash_len', 'Original_len', 'signature_len', 'From_len', 'To_len',
    'sender_len', 'paymaster_len', 'Txn Fee', 'logIndex', 'actualGasCost',
    'actualGasUsed', 'nonce', 'success', 'Blockno', 'DateTime_ts'
]
target = 'Gas Used'

X_raw = df[features].copy()
y_raw = df[target].astype(float).copy()

# Nếu bạn cần hold-out cho Hyperopt, giữ block sau; không dùng ở Repeated K-Fold
X_train, X_val, y_train, y_val = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=SEED
)
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_train_s = x_scaler.fit_transform(X_train)
X_val_s   = x_scaler.transform(X_val)
y_train_s = y_scaler.fit_transform(y_train.values.reshape(-1,1)).ravel()
y_val_s   = y_scaler.transform(y_val.values.reshape(-1,1)).ravel()

# DỮ LIỆU DÙNG CHO REPEATED K-FOLD (toàn bộ tập)
X = X_raw.copy()
y_trainable = y_raw.copy()

# ================== 2) REPEATED K-FOLD ==================
cats = np.zeros((EPOCHS, 4))  # [RMSE, MSE, MAE, R2] trên thang MinMax-scaled

for ep in range(EPOCHS):
    kf_rep = KFold(n_splits=K, shuffle=True, random_state=SEED + ep)

    rmse_list, mse_list, mae_list, r2_list = [], [], [], []

    for fold, (tr_idx, va_idx) in enumerate(kf_rep.split(X), start=1):
        X_tr, X_va = X.iloc[tr_idx].values, X.iloc[va_idx].values
        y_tr, y_va = y_trainable.iloc[tr_idx].values, y_trainable.iloc[va_idx].values

        # SCALE trong từng fold để tránh leakage
        sx = MinMaxScaler()
        X_tr_s = sx.fit_transform(X_tr)
        X_va_s = sx.transform(X_va)

        sy = MinMaxScaler()
        y_tr_s = sy.fit_transform(y_tr.reshape(-1, 1)).ravel()
        y_va_s = sy.transform(y_va.reshape(-1, 1)).ravel()

        # Build model (thử rsm, nếu lỗi thì bỏ rsm)
        kwargs = build_kwargs(ep)
        try:
            model = CatBoostRegressor(**kwargs)
        except Exception as e:
            if "Unknown parameter" in str(e) and "rsm" in str(e):
                kwargs.pop('rsm', None)
                model = CatBoostRegressor(**kwargs)
            else:
                raise

        model.fit(
            X_tr_s, y_tr_s,
            eval_set=(X_va_s, y_va_s),
            early_stopping_rounds=200,
            use_best_model=True,
            verbose=False
        )

        preds_s = model.predict(X_va_s)

        # Metrics trên thang MinMax-scaled của y
        mse  = mean_squared_error(y_va_s, preds_s)
        rmse = float(np.sqrt(mse))
        mae  = mean_absolute_error(y_va_s, preds_s)
        r2   = r2_score(y_va_s, preds_s)

        rmse_list.append(rmse)
        mse_list.append(mse)
        mae_list.append(mae)
        r2_list.append(r2)

    # Lưu mean metrics của epoch này (qua K folds)
    cats[ep, :] = [
        np.mean(rmse_list),
        np.mean(mse_list),
        np.mean(mae_list),
        np.mean(r2_list)
    ]
    print(f"Epoch {ep+1:02d}/{EPOCHS} -> RMSE={cats[ep,0]:.6f} | MSE={cats[ep,1]:.6f} | MAE={cats[ep,2]:.6f} | R2={cats[ep,3]:.6f}")

# ================== 3) TRUNG BÌNH CUỐI ==================
final_mean = np.mean(cats, axis=0)
print("\nFinal mean on MinMax-scaled y [RMSE, MSE, MAE, R2]:")
print(final_mean)
print("\nPer-epoch metrics (rows) on MinMax-scaled y [RMSE, MSE, MAE, R2]:")
print(cats)

Epoch 01/20 -> RMSE=0.004404 | MSE=0.000026 | MAE=0.000726 | R2=0.952829
Epoch 02/20 -> RMSE=0.005114 | MSE=0.000033 | MAE=0.000741 | R2=0.943892
Epoch 03/20 -> RMSE=0.004888 | MSE=0.000028 | MAE=0.000729 | R2=0.949204
Epoch 04/20 -> RMSE=0.004679 | MSE=0.000026 | MAE=0.000804 | R2=0.952704
Epoch 05/20 -> RMSE=0.004558 | MSE=0.000026 | MAE=0.000692 | R2=0.953883
Epoch 06/20 -> RMSE=0.004767 | MSE=0.000026 | MAE=0.000695 | R2=0.951835
Epoch 07/20 -> RMSE=0.004557 | MSE=0.000025 | MAE=0.000693 | R2=0.954250
Epoch 08/20 -> RMSE=0.005137 | MSE=0.000030 | MAE=0.000733 | R2=0.942760
Epoch 09/20 -> RMSE=0.004938 | MSE=0.000028 | MAE=0.000748 | R2=0.947127
Epoch 10/20 -> RMSE=0.004408 | MSE=0.000025 | MAE=0.000697 | R2=0.958188
Epoch 11/20 -> RMSE=0.004250 | MSE=0.000022 | MAE=0.000682 | R2=0.961882
Epoch 12/20 -> RMSE=0.004604 | MSE=0.000027 | MAE=0.000689 | R2=0.954734
Epoch 13/20 -> RMSE=0.004745 | MSE=0.000027 | MAE=0.000701 | R2=0.951734
Epoch 14/20 -> RMSE=0.004824 | MSE=0.000028 | MAE=0